In [18]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.functions import col, sum, dense_rank, round
from pyspark.sql.window import Window

In [19]:
# Spark Configuration
sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("Top_Categories_Products_Pipeline")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

# Setup hadoop fs configuration for schema gs://
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

data_bucket_uri = "data_de2024_a2"
temp_bucket = "temp_de2024_mh"
project_id = "core-synthesis-435410-v9"

In [20]:
# Load Fact and Dimension Tables
factDF = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(f"gs://{data_bucket_uri}/fact_table.csv")
itemDF = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(f"gs://{data_bucket_uri}/item_dim.csv")
storeDF = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(f"gs://{data_bucket_uri}/store_dim.csv")
transDF = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(f"gs://{data_bucket_uri}/Trans_dim.csv")
timeDF = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(f"gs://{data_bucket_uri}/time_dim.csv")
customerDF = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(f"gs://{data_bucket_uri}/customer_dim.csv")

In [21]:
# Drop eventual duplicates
itemDF_cleaned = itemDF.drop("unit_price", "unit_price")
# Join Fact Table with Item and Store Dimension Tables
joinedDF = factDF.join(itemDF_cleaned, "item_key").join(storeDF, "store_key").join(timeDF, "time_key").join(transDF, "payment_key").join(customerDF, "coustomer_key")

print(joinedDF.printSchema())
joinedDF.show()

root
 |-- coustomer_key: string (nullable = true)
 |-- payment_key: string (nullable = true)
 |-- time_key: string (nullable = true)
 |-- store_key: string (nullable = true)
 |-- item_key: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- unit: string (nullable = true)
 |-- unit_price: double (nullable = true)
 |-- total_price: double (nullable = true)
 |-- item_name: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- man_country: string (nullable = true)
 |-- supplier: string (nullable = true)
 |-- unit: string (nullable = true)
 |-- division: string (nullable = true)
 |-- district: string (nullable = true)
 |-- upazila: string (nullable = true)
 |-- date: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: string (nullable = true)
 |-- month: integer (nullable = true)
 |-- quarter: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- trans_type: string (nullable = true)
 |-- ban

In [22]:
# Aggregate sales, quantity, and calculate average price per item
store_category_sales = joinedDF.groupBy("store_key", "desc").agg(
    sum("total_price").alias("category_sales"),
    sum("quantity").alias("total_quantity")
).withColumn(
    "avg_price_per_item", round(col("category_sales") / col("total_quantity"))
)

# Calculate total sales per store
store_total_sales = joinedDF.groupBy("store_key") \
    .sum("total_price") \
    .withColumnRenamed("sum(total_price)", "store_sales")

# Join and calculate contribution percentage
store_category_contribution = store_category_sales.join(
    store_total_sales, on="store_key"
).withColumn(
    "contribution_pct", round((col("category_sales") / col("store_sales")) * 100,2)
)

# Identify the top-contributing categories for each store using window func and rank
window = Window.partitionBy("store_key").orderBy(col("category_sales").desc())
top_categories_per_store = store_category_contribution.withColumn(
    "rank", dense_rank().over(window)
)

# Take top category (rank == 1)
top_category_per_store = top_categories_per_store.filter(col("rank") == 1).select(
    "store_key", "desc", "category_sales", "contribution_pct", "avg_price_per_item"
)

top_category_per_store.show()

+---------+--------------------+--------------+----------------+------------------+
|store_key|                desc|category_sales|contribution_pct|avg_price_per_item|
+---------+--------------------+--------------+----------------+------------------+
|    S0001|      Food - Healthy|       15181.0|           10.77|              15.0|
|    S0002|      Food - Healthy|       16999.5|           11.59|              16.0|
|    S0003|Beverage - Energy...|       15396.0|           10.58|              33.0|
|    S0004|      Food - Healthy|       14009.5|           10.35|              16.0|
|    S0005|Beverage - Energy...|       15675.0|           10.72|              35.0|
|    S0006|Beverage - Energy...|       14487.0|            9.71|              36.0|
|    S0007|Beverage - Energy...|       14585.0|           10.02|              34.0|
|    S0008|Beverage - Energy...|       16729.0|           11.57|              35.0|
|    S0009|Beverage - Energy...|       15213.0|           10.23|            

In [23]:
# Save to BigQuery
spark.conf.set('temporaryGcsBucket', temp_bucket)

top_category_per_store.write.format('bigquery') \
    .option('table', f'{project_id}.a2.top_product_categories_by_store') \
    .mode("overwrite") \
    .save()

# Stop Spark Session
spark.stop()